In [ ]:

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pickle


In [ ]:
#Load dataset (Replace with actual dataset path)
df = pd.read_csv("customer_churn.csv")

# Data Preprocessing
df.drop_duplicates(inplace=True)
df.fillna(df.median(), inplace=True)

# Encoding categorical variables
encoder = LabelEncoder()
categorical_columns = ['Gender', 'PaymentMethod', 'Contract']
for col in categorical_columns:
    df[col] = encoder.fit_transform(df[col])

# Feature Scaling
scaler = StandardScaler()
numerical_columns = ['tenure', 'MonthlyCharges', 'TotalCharges']
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

# Splitting dataset
X = df.drop('Churn', axis=1)  # Replace 'Churn' with actual target column
y = df['Churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:

# Model Selection and Training
model_rf = RandomForestClassifier()
model_svm = SVC()
model_rf.fit(X_train, y_train)
model_svm.fit(X_train, y_train)


In [ ]:

# Hyperparameter Tuning for Random Forest
params = {'n_estimators': [100, 200], 'max_depth': [5, 10]}
gs = GridSearchCV(RandomForestClassifier(), params, cv=5)
gs.fit(X_train, y_train)
print("Best Parameters:", gs.best_params_)


In [ ]:

# Model Evaluation
pred_rf = model_rf.predict(X_test)
pred_svm = model_svm.predict(X_test)

print("RandomForest Accuracy:", accuracy_score(y_test, pred_rf))
print("SVM Accuracy:", accuracy_score(y_test, pred_svm))
print("RandomForest Classification Report:\n", classification_report(y_test, pred_rf))


In [ ]:

# Save the best model
with open('best_model.pkl', 'wb') as f:
    pickle.dump(model_rf, f)


In [ ]:

# Deployment using Streamlit
import streamlit as st

st.title("Customer Churn Prediction")
model = pickle.load(open('best_model.pkl', 'rb'))

user_input = st.text_input("Enter input values (comma-separated):")

if st.button("Predict"):
    input_data = np.array([float(i) for i in user_input.split(',')]).reshape(1, -1)
    prediction = model.predict(input_data)
    st.write("Prediction:", "Churn" if prediction[0] == 1 else "No Churn")
